# Algoritmo para escribir un excel


## Importación de librerias necesarias


In [1]:
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import (
    PatternFill,
    Border,
    Side,
    Alignment,
    Protection,
    Font,
    Color,
)
from openpyxl.utils.cell import get_column_letter, coordinate_from_string
import datetime
import piexif
import pandas as pd

from PIL import Image as PILImage

## Funciones necesarias

In [2]:
def reducir_tamano_imagen(
    archivo_entrada, archivo_salida, nueva_resolucion=(1920, 1080), calidad=85
):
    # Abrir la imagen
    imagen = PILImage.open(archivo_entrada)

    # Reducir la resolución
    imagen = imagen.resize(nueva_resolucion)

    # Eliminar metadatos
    imagen_info = imagen.info
    imagen_exif = imagen_info.get("exif")
    if imagen_exif:
        nueva_exif = piexif.load(imagen_exif)
        nueva_exif.pop("thumbnail", None)  # Eliminar miniatura si existe
        nueva_exif_bytes = piexif.dump(nueva_exif)
        imagen_info["exif"] = nueva_exif_bytes

    # Guardar la imagen reducida
    imagen.save(archivo_salida, quality=calidad)

    # Cerrar la imagen original
    imagen.close()


# Uso del ejemplo
# reducir_tamano_imagen('../images/60.JPG', '../images-low-quality/60-low-quality.JPG')

In [3]:
def escalar_y_guardar_imagen(ruta_imagen, escala_factor, ruta_guardado):
    """
    Escala una imagen y la guarda en una nueva ubicación.

    Parámetros:
    - ruta_imagen: La ruta de la imagen a escalar.
    - escala_factor: El factor de escala para reducir o aumentar la imagen.
    - ruta_guardado: La ruta donde se guardará la imagen escalada.

    """
    # Abrir la imagen
    imagen = PILImage.open(ruta_imagen)

    # Calcular las nuevas dimensiones
    nuevo_ancho = int(imagen.width * escala_factor)
    nuevo_alto = int(imagen.height * escala_factor)

    # Escalar la imagen
    imagen_escalada = imagen.resize((nuevo_ancho, nuevo_alto))

    # Guardar la imagen
    imagen_escalada.save(ruta_guardado)

In [4]:
def convertir_a_referencia_excel(fila, columna):
    # Convertir números de fila y columna a referencia de celda
    referencia_excel = get_column_letter(columna) + str(fila)

    return referencia_excel


def convertir_desde_referencia_excel(referencia_excel):
    # Convertir referencia de celda a números de fila y columna
    columna, fila = coordinate_from_string(referencia_excel)

    return fila, columna


# Ejemplos de uso
print(convertir_a_referencia_excel(1, 1))  # Salida: "A1"
print(convertir_a_referencia_excel(2, 26))  # Salida: "BZ2"
print(convertir_a_referencia_excel(10, 5))  # Salida: "E10"

# Ejemplo de conversión desde referencia de celda a números de fila y columna
print(convertir_desde_referencia_excel("A1"))  # Salida: (1, 1)
print(convertir_desde_referencia_excel("BZ2"))  # Salida: (2, 26)
print(convertir_desde_referencia_excel("E10"))  # Salida: (10, 5)

A1
Z2
E10
(1, 'A')
(2, 'BZ')
(10, 'E')


In [5]:
def formato_progresiva(valor_en_metros):
    km, metros = divmod(valor_en_metros, 1000)
    return f"{int(km)}+{metros:03d}"

## Input


In [6]:
titulo = "4.- FICHA FOTOS DEL ITINERARIO DE LA CARRETERA"

subtitulo = "Panel Fotográfico"

In [7]:
df = pd.read_excel("./data/progresivas-mandraqueadas.xlsx", sheet_name="Hoja1")
df.head()

,progresivas,ancho,largo,descripcion_formal,descripcion_personalizada,progresivas_al_reves,id_foto,isValid
0,0,4,-,-,-,10872,68.0,1
1,180,5,-,Alcantarilla,alcantarilla de dos ojos,10620,67.0,1
2,250,5,-,-,-,10570,66.0,1
3,500,5,-,-,-,10570,65.0,0
4,750,4,-,-,-,10320,64.0,1


## Filtrando valores que no son validos


In [8]:
filtered_df = df.loc[lambda x: x["isValid"] == 1]
filtered_df=filtered_df.iloc[::-1]
filtered_df['descripcion_formal']=filtered_df['descripcion_formal'].replace('-','Medición')

In [9]:
filtered_df.head()

,progresivas,ancho,largo,descripcion_formal,descripcion_personalizada,progresivas_al_reves,id_foto,isValid
69,11070,-,-,Hito_kilometrico,Hito 00,0,0.0,1
68,11050,4,10.5,Inicio,Puente,20,1.0,1
67,11020,-,-,Señal_informativa,Señal,50,2.0,1
66,11000,4.6,-,Medición,-,70,3.0,1
65,10750,4.2,-,Medición,-,320,4.0,1


## Diviendo el dataframe en grupos de 6


In [10]:
tamanio_subconjunto = 6

num_subconjuntos = len(filtered_df) // tamanio_subconjunto + (
    len(filtered_df) % tamanio_subconjunto > 0
)

lista_de_dataframes = [filtered_df.iloc[i * tamanio_subconjunto: (i + 1) * tamanio_subconjunto] for i in range(num_subconjuntos)]


In [11]:
lista_de_cardLists=[]

for i, subconjunto in enumerate(lista_de_dataframes):
    new_cardList = []
    for j, row in subconjunto.iterrows():

        new_cardList.append(
            {
                "id": j,
                "description": f"{row['descripcion_formal']}",
                "progresivve": formato_progresiva(row['progresivas_al_reves']),
                "image_path": f"./images-low-quality/{row['id_foto']}-low-quality.JPG",
            }
        )
    lista_de_cardLists.append(new_cardList)

## Seteando variables iniciales


In [12]:
path_mtc_low_quality = "./assets-low-quality/mtc-low-quality.jpg"
escalar_y_guardar_imagen(
    "./assets/mtc.jpg",
    escala_factor=0.9,
    ruta_guardado=path_mtc_low_quality,
)
## La imagen escalada del mtc tiene el alto de 3 filas

## Creando un workbook


In [13]:
wb = Workbook()
ws = wb.active

ancho_columna = 48

ws.column_dimensions["A"].width = 2
ws.column_dimensions["B"].width = ancho_columna
ws.column_dimensions["C"].width = 1
ws.column_dimensions["D"].width = ancho_columna

## Creando el indice para definir donde insertar el siguiente bloque de excel


In [14]:
global row_index
row_index = 1

In [15]:
# 4000 pixeles de ancho equivalen a 50 celdas de excel, donde cada celda tiene 10.71 unidades
# 3000 pixeles de alto equivalen a 101 celdas de excel, donde cada celda tiene 15 unidades

alto_celda_equivalencia = 3000 / (101 * 15)
ancho_celda_equivalencia = 4000 / (50 * 10.71)
print(alto_celda_equivalencia)
print(ancho_celda_equivalencia)

borde = Border(
    left=Side(border_style="thin"),
    right=Side(border_style="thin"),
    top=Side(border_style="thin"),
    bottom=Side(border_style="thin"),
)


def write_card(number, description, progresivve, image_path, row, column):
    imagen = Image(image_path)
    altura_celda_1 = 195
    altura_celda_2 = 45
    ws.row_dimensions[row].height = altura_celda_1

    alto_celda_1_pixeles = altura_celda_1 * alto_celda_equivalencia
    alto_celda_2_pixeles = altura_celda_2 * alto_celda_equivalencia
    ancho_celda_pixeles = ancho_celda_equivalencia * ancho_columna

    factor_correccion_escala_alto = 0.65
    factor_correccion_escala_ancho = 0.9

    if imagen.height > imagen.width:
        # print("Imagen vertical")
        factor_escala = alto_celda_1_pixeles / imagen.height
        imagen.width *= factor_escala
        imagen.height = alto_celda_1_pixeles

        imagen.width *= factor_correccion_escala_alto
        imagen.height *= factor_correccion_escala_alto
    else:
        # print("Imagen horizontal")
        factor_escala = ancho_celda_pixeles / imagen.width
        imagen.width = ancho_celda_pixeles
        imagen.height *= factor_escala

        imagen.width *= factor_correccion_escala_ancho
        imagen.height *= factor_correccion_escala_ancho
    ws.add_image(imagen, f"{column}{row}")
    ws[f"{column}{row}"].border = borde

    row += 1
    ws[f"{column}{row}"] = f"Foto {number}: {description} - {progresivve}"
    ws[f"{column}{row}"].border = borde
    ws[f"{column}{row}"].alignment = Alignment(vertical="center")
    ws.row_dimensions[row].height = altura_celda_2

1.9801980198019802
7.469654528478058


## Contador de fotos

In [16]:
global photo_number
photo_number=1

In [17]:
def page_inserter(cardList):
    global row_index
    global photo_number
    row_index += 1

    img = Image(path_mtc_low_quality)
    ws.add_image(img, f"B{row_index}")
    row_index += 3

    row_index += 1

    ## Unimos las celdas para el titulo

    ws.merge_cells(f"B{row_index}:D{row_index}")

    ## Escribimos el titulo

    ws[f"B{row_index}"] = titulo

    ## Centramos el titulo y le damos formato negrita y tamanio 14

    ws[f"B{row_index}"].alignment = Alignment(horizontal="center")
    ws[f"B{row_index}"].font = Font(bold=True, size=14)

    row_index += 2

    ## Unimos las celdas para el subtitulo

    ws.merge_cells(f"B{row_index}:D{row_index}")

    ## Escribimos el subtitulo

    ws[f"B{row_index}"] = subtitulo

    ## Centramos el subtitulo y le damos formato negrita y tamanio 14

    ws[f"B{row_index}"].alignment = Alignment(horizontal="center")
    ws[f"B{row_index}"].font = Font(bold=True, size=12)

    row_index += 2

    for i in range(0, len(cardList), 2):
        write_card(
            photo_number,
            cardList[i]["description"],
            cardList[i]["progresivve"],
            cardList[i]["image_path"],
            row_index,
            "B",
        )
        photo_number+=1
        # row_index += 2
        if i + 1 < len(cardList):
            write_card(
                photo_number,
                cardList[i+1]["description"],
                cardList[i+1]["progresivve"],
                cardList[i+1]["image_path"],
                row_index,
                "D",
            )
            photo_number+=1
            row_index += 3

In [18]:
for cardList in lista_de_cardLists:
    page_inserter(cardList)
    row_index += 2

FileNotFoundError: [Errno 2] No such file or directory: './images-low-quality/0.0-low-quality.JPG'

## Guardando el libro en un archivo de trabajo


In [ ]:
wb.save("./excel/ficha_fotos.xlsx")